In this assignment, you'll work on transparency of Bernoulli naive Bayes. You need a "binary classification" dataset (two classes), and you need all features to be either already binary or you need to binarize them yourself. Do NOT use the "binarize" field in the constructor of BernoulliNB. Each continuous feature should be binarized based on whether it is larger or smaller than its mean. Categorical features that have more than two possible values should use one-hot encoding.  The new features should have proper names according to the binarization. Ask me if you are uncertain about any of this.

1. Choose a binary classification dataset. (The same rules in Assignment 3 applies, except there is one additional rule this time: it has to be a classification dataset).

2. Create a Jupyter notebook. Name it assignment8. It should have several sections. The first section should describe the dataset. The second section should load the dataset. The third section should binarize all the features.

3. If the dataset already comes in train-test split, great. If not, create a train-test split, using 2/3 for train and 1/3 for test.

4. Train a BernouilliNB classifier on the train split; use the default parameters.

5. For the following types of objects, create a section, and print a) the total positive log-evidence,  b) the total negative log-evidence, c) probability distribution, d) top 3 features values that contribute most to the positive evidence, e) top 3 feature values that contribute the most to the negative evidence. Print this information on the following object types
    - The most positive object with respect to the probabilities.
    - The most negative object with respect to the probabilities.
    - The object that has the largest positive evidence.
    - The object that has the largest (in magnitude) negative evidence.
    - The most uncertain object (the probabilities are closest to 0.5)

Upload the notebook to your 595 github repository.

Submit the link using blackboard.